**Import libraries**

In [50]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import requests

**Data Acquisition**

**API ABASTORES**

In [51]:
base_url = "https://api.abastores.com/api/v2/marketdata/dataprice-rag/"

# Inicializar una lista para almacenar los datos de todas las páginas
all_data = []

# Iterar sobre las 36 páginas
for page_number in range(1, 38):
    # Construir el URL completo para la página actual
    url = f"{base_url}?page={page_number}&page_size=1000"
    
    # Enviar una solicitud GET al endpoint
    response = requests.get(url)
    
    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Obtener los datos de la respuesta
        data = response.json()
        
        # Extraer valores de la columna "results" y crear columnas separadas
        df = pd.json_normalize(data['results'])
        
        # Agregar el DataFrame actual a la lista
        all_data.append(df)
    else:
        print(f"Failed to retrieve data for page {page_number}. Status code:", response.status_code)

# Concatenar todos los DataFrames en un solo DataFrame
final_df = pd.concat(all_data, ignore_index=True)

/var/folders/13/vtwv0p951k11cf7tvhd750jw0000gn/T/ipykernel_1090/1855668934.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_data, ignore_index=True)


In [128]:
#final_df.to_csv('abastores_data_prices.csv')

**Copy dataframe**

In [111]:
df_ = final_df.copy()

**Explore columns**

In [112]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36439 entries, 0 to 36438
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         36439 non-null  int64  
 1   date                       36439 non-null  object 
 2   price                      36439 non-null  float64
 3   quantity                   27745 non-null  float64
 4   region                     27711 non-null  float64
 5   product.id                 36439 non-null  int64  
 6   product.meta_product.id    36439 non-null  int64  
 7   product.meta_product.name  36439 non-null  object 
 8   product.meta_product.icon  34929 non-null  object 
 9   product.family.id          36439 non-null  int64  
 10  product.family.name        36439 non-null  object 
 11  product.variety.id         36439 non-null  int64  
 12  product.variety.name       36439 non-null  object 
 13  data_source.id             36439 non-null  int

**Look for missing values**

In [113]:
missing_values_absolute = df_.isna().sum()
missing_values_percentage = (df_.isna().mean()) * 100
missing_info = pd.DataFrame({'Missing Values': missing_values_absolute, 'Percentage': missing_values_percentage})
missing_info

,Missing Values,Percentage
id,0,0.000000
date,0,0.000000
price,0,0.000000
quantity,8694,23.859052
region,8728,23.952359
product.id,0,0.000000
product.meta_product.id,0,0.000000
product.meta_product.name,0,0.000000
product.meta_product.icon,1510,4.143912
product.family.id,0,0.000000


**We create a new dataframe with the columns we are going to use**

In [114]:
df_ = df_[['date','product.meta_product.name','price','data_source.name','province.name']]

In [115]:
df_

,date,product.meta_product.name,price,data_source.name,province.name
0,2024-05-07T17:25:00+02:00,Maíz,210.00,Abastores,León
1,2024-05-07T17:25:00+02:00,Maíz,218.00,Abastores,Albacete
2,2024-05-07T17:25:00+02:00,Cebada,197.00,Abastores,Palencia
3,2024-05-07T17:25:00+02:00,Cebada,197.00,Abastores,Zaragoza
4,2024-05-07T17:25:00+02:00,Cebada,195.00,Abastores,Lleida
...,...,...,...,...,...
36434,2002-01-04T01:00:00+01:00,Trigo,180.30,Lonja de Toledo,Toledo
36435,2002-01-04T01:00:00+01:00,Trigo,147.25,Lonja de Toledo,Toledo
36436,2002-01-04T01:00:00+01:00,Trigo,171.29,Lonja de Toledo,Toledo
36437,2002-01-04T01:00:00+01:00,Trigo,153.26,Lonja de Toledo,Toledo


**We convert date to datetime format an extract the date without time**

In [116]:
# Convertir la columna 'date' al tipo de datos datetime
df_['date'] = pd.to_datetime(df_['date'],utc=True,format='mixed')

# Extraer solo la fecha de la columna 'date'
df_['date'] = df_['date'].dt.date

In [117]:
df_

,date,product.meta_product.name,price,data_source.name,province.name
0,2024-05-07,Maíz,210.00,Abastores,León
1,2024-05-07,Maíz,218.00,Abastores,Albacete
2,2024-05-07,Cebada,197.00,Abastores,Palencia
3,2024-05-07,Cebada,197.00,Abastores,Zaragoza
4,2024-05-07,Cebada,195.00,Abastores,Lleida
...,...,...,...,...,...
36434,2002-01-04,Trigo,180.30,Lonja de Toledo,Toledo
36435,2002-01-04,Trigo,147.25,Lonja de Toledo,Toledo
36436,2002-01-04,Trigo,171.29,Lonja de Toledo,Toledo
36437,2002-01-04,Trigo,153.26,Lonja de Toledo,Toledo


**Divide date in different columns**

In [118]:
df_['dia']=pd.to_datetime(df_['date']).dt.day
df_['mes']=pd.to_datetime(df_['date']).dt.month
df_['anio']=pd.to_datetime(df_['date']).dt.year

**Now lets feed the rag**

In [119]:
df_['date'].max()

datetime.date(2024, 5, 7)

In [120]:
# Define mappings
dia_mapping = {
    1: 'uno',
    2: 'dos',
    3: 'tres',
    4: 'cuatro',
    5: 'cinco',
    6: 'seis',
    7: 'siete',
    8: 'ocho',
    9: 'nueve',
    10: 'diez',
    11: 'once',
    12: 'doce',
    13: 'trece',
    14: 'catorce',
    15: 'quince',
    16: 'dieciséis',
    17: 'diecisiete',
    18: 'dieciocho',
    19: 'diecinueve',
    20: 'veinte',
    21: 'veintiuno',
    22: 'veintidós',
    23: 'veintitrés',
    24: 'veinticuatro',
    25: 'veinticinco',
    26: 'veintiséis',
    27: 'veintisiete',
    28: 'veintiocho',
    29: 'veintinueve',
    30: 'treinta',
    31: 'treinta y uno'
}

mes_mapping = {
    1: 'enero',
    2: 'febrero',
    3: 'marzo',
    4: 'abril',
    5: 'mayo',
    6: 'junio',
    7: 'julio',
    8: 'agosto',
    9: 'septiembre',
    10: 'octubre',
    11: 'noviembre',
    12: 'diciembre'
}

anio_mapping = {
    2000: 'dos mil',
    2001: 'dos mil uno',
    2002: 'dos mil dos',
    2003: 'dos mil tres',
    2004: 'dos mil cuatro',
    2005: 'dos mil cinco',
    2006: 'dos mil seis',
    2007: 'dos mil siete',
    2008: 'dos mil ocho',
    2009: 'dos mil nueve',
    2010: 'dos mil diez',
    2011: 'dos mil once',
    2012: 'dos mil doce',
    2013: 'dos mil trece',
    2014: 'dos mil catorce',
    2015: 'dos mil quince',
    2016: 'dos mil dieciséis',
    2017: 'dos mil diecisiete',
    2018: 'dos mil dieciocho',
    2019: 'dos mil diecinueve',
    2020: 'dos mil veinte',
    2021: 'dos mil veintiuno',
    2022: 'dos mil veintidós',
    2023: 'dos mil veintitrés',
    2024: 'dos mil veinticuatro'
}

# Function to convert date components to strings
def convert_to_strings(row):
    dia_str = dia_mapping[row['dia']]
    mes_str = mes_mapping[row['mes']]
    anio_str = anio_mapping[row['anio']]
    return pd.Series([dia_str, mes_str, anio_str])

df_['dia_str']=''
df_['mes_str']=''
df_['anio_str']=''

# Apply the conversion function to each row
df_[['dia_str', 'mes_str', 'anio_str']] = df_[['dia', 'mes', 'anio']].apply(convert_to_strings, axis=1)

In [121]:
df_['first_line']=''
# Crear la columna deseada con la estructura especificada
df_['first_line'] = df_.apply(lambda row: 
    f'precio: {row["price"]} fecha: {row["date"]}', 
    axis=1)

In [171]:
df_[df_['product.meta_product.name']=='Maíz']

,date,product.meta_product.name,price,data_source.name,province.name,dia,mes,anio,dia_str,mes_str,anio_str,first_line
0,2024-05-07,Maíz,210.00,Abastores,León,7,5,2024,siete,mayo,dos mil veinticuatro,precio: 210.0 fecha: 2024-05-07
1,2024-05-07,Maíz,218.00,Abastores,Albacete,7,5,2024,siete,mayo,dos mil veinticuatro,precio: 218.0 fecha: 2024-05-07
10,2024-05-07,Maíz,172.00,Bolsa de Chicago,Estados Unidos,7,5,2024,siete,mayo,dos mil veinticuatro,precio: 172.0 fecha: 2024-05-07
16,2024-05-07,Maíz,208.25,Euronext,París,7,5,2024,siete,mayo,dos mil veinticuatro,precio: 208.25 fecha: 2024-05-07
17,2024-05-07,Maíz,214.00,Lonja de Sevilla,Sevilla,7,5,2024,siete,mayo,dos mil veinticuatro,precio: 214.0 fecha: 2024-05-07
...,...,...,...,...,...,...,...,...,...,...,...,...
36394,2002-01-18,Maíz,144.24,Lonja de Toledo,Toledo,18,1,2002,dieciocho,enero,dos mil dos,precio: 144.24 fecha: 2002-01-18
36403,2002-01-11,Maíz,144.24,Lonja de Toledo,Toledo,11,1,2002,once,enero,dos mil dos,precio: 144.24 fecha: 2002-01-11
36412,2002-01-11,Maíz,144.24,Lonja de Toledo,Toledo,11,1,2002,once,enero,dos mil dos,precio: 144.24 fecha: 2002-01-11
36421,2002-01-04,Maíz,141.24,Lonja de Toledo,Toledo,4,1,2002,cuatro,enero,dos mil dos,precio: 141.24 fecha: 2002-01-04


In [123]:
# Agrupar por mes y año y concatenar los textos de sentence_chunk
grouped_chunks = df_.groupby(['mes', 'anio','mes_str','anio_str','product.meta_product.name','data_source.name','province.name']).agg({
    'first_line': ' '.join,
}).reset_index()

In [124]:
grouped_chunks['title']=''
# Crear la columna deseada con la estructura especificada
grouped_chunks['title'] = grouped_chunks.apply(lambda row: 
    f'Provincia: {row["province.name"]} '
    f'Producto: {row["product.meta_product.name"]} '
    f'Fuente de datos: {row["data_source.name"]} '
    f'Fecha: mes: {row["mes"]}-{row["mes_str"]} '
    f'año: {row["anio"]}-{row["anio_str"]}',
    axis=1)

**We have one title to identify the data we need**

In [125]:
grouped_chunks['title'][0]

'Provincia: Toledo Producto: Avena Fuente de datos: Lonja de Toledo Fecha: mes: 1-enero año: 2002-dos mil dos'

**first_line will be the data that we are going to ingest into the LLM**

In [126]:
grouped_chunks['first_line'][0]

'precio: 153.0 fecha: 2002-01-25 precio: 153.0 fecha: 2002-01-25 precio: 153.25 fecha: 2002-01-18 precio: 153.25 fecha: 2002-01-18 precio: 153.25 fecha: 2002-01-11 precio: 153.25 fecha: 2002-01-11 precio: 150.25 fecha: 2002-01-04 precio: 150.25 fecha: 2002-01-04'

In [131]:
grouped_chunks.columns

Index(['mes', 'anio', 'mes_str', 'anio_str', 'product.meta_product.name',
       'data_source.name', 'province.name', 'first_line', 'title'],
      dtype='object')

In [132]:
df_chunk=grouped_chunks[['title','first_line','mes','anio','province.name','product.meta_product.name']]

**Structure the information**

In [149]:
pages_and_chunks = []

for index, row in tqdm(df_chunk.iterrows(), total=len(df_chunk)):
    chunk_dict = {}
    joined_sentence_chunk = row['title'].replace(" ", " ").strip()
    joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
    
    chunk_dict["sentence_chunk"] = joined_sentence_chunk
    chunk_dict["first_line"] = row["first_line"]
    chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
    chunk_dict["chunk_word_count"] = len(joined_sentence_chunk.split(" "))
    chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4  # 1 token =~ 4 chars

    # Add 'mes' and 'anio' columns from df_chunk to chunk_dict
    chunk_dict["mes"] = row["mes"]
    chunk_dict["anio"] = row["anio"]
    chunk_dict["producto"] = row["product.meta_product.name"]
    chunk_dict["provincia"] = row["province.name"]
    

    pages_and_chunks.append(chunk_dict)


100%|██████████| 3917/3917 [00:00<00:00, 41081.97it/s]


In [150]:
pages_and_chunks

[{'sentence_chunk': 'Provincia: Toledo Producto: Avena Fuente de datos: Lonja de Toledo Fecha: mes: 1-enero año: 2002-dos mil dos',
  'first_line': 'precio: 153.0 fecha: 2002-01-25 precio: 153.0 fecha: 2002-01-25 precio: 153.25 fecha: 2002-01-18 precio: 153.25 fecha: 2002-01-18 precio: 153.25 fecha: 2002-01-11 precio: 153.25 fecha: 2002-01-11 precio: 150.25 fecha: 2002-01-04 precio: 150.25 fecha: 2002-01-04',
  'chunk_char_count': 108,
  'chunk_word_count': 17,
  'chunk_token_count': 27.0,
  'mes': 1,
  'anio': 2002,
  'producto': 'Avena',
  'provincia': 'Toledo'},
 {'sentence_chunk': 'Provincia: Toledo Producto: Cebada Fuente de datos: Lonja de Toledo Fecha: mes: 1-enero año: 2002-dos mil dos',
  'first_line': 'precio: 143.0 fecha: 2002-01-25 precio: 144.0 fecha: 2002-01-25 precio: 143.0 fecha: 2002-01-25 precio: 144.0 fecha: 2002-01-25 precio: 144.23 fecha: 2002-01-18 precio: 144.24 fecha: 2002-01-18 precio: 144.23 fecha: 2002-01-18 precio: 144.24 fecha: 2002-01-18 precio: 144.23 fec

**Embedding our text chunks**

In [145]:
!pip install -U sentence-transformers

In [146]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", device="cpu")

/Users/daniel/anaconda3/envs/test_environment/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [151]:
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks]
text_chunks[0]

'Provincia: Toledo Producto: Avena Fuente de datos: Lonja de Toledo Fecha: mes: 1-enero año: 2002-dos mil dos'

In [152]:
len(text_chunks)

3917

In [153]:
%%time

# Embed all texts in batches
text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=30,#You can experiment to find which batch size leads to best results
                                               convert_to_tensor=True)

text_chunk_embeddings

CPU times: user 5min 9s, sys: 2min 4s, total: 7min 13s
Wall time: 1min 1s


tensor([[-0.0254, -0.0132, -0.0287,  ...,  0.0015, -0.0529, -0.0130],
        [-0.0201, -0.0133, -0.0268,  ..., -0.0003, -0.0521, -0.0113],
        [-0.0265, -0.0201, -0.0267,  ...,  0.0004, -0.0494, -0.0122],
        ...,
        [ 0.0291, -0.0129, -0.0127,  ...,  0.0033, -0.0356,  0.0138],
        [ 0.0048, -0.0093, -0.0279,  ...,  0.0195, -0.0401, -0.0063],
        [ 0.0407, -0.0320, -0.0344,  ...,  0.0020, -0.0210, -0.0305]])

In [154]:
for item in tqdm(pages_and_chunks):
    item['embedding'] = embedding_model.encode(item["sentence_chunk"])

100%|██████████| 3917/3917 [02:35<00:00, 25.12it/s]


**Save information in the csv (in this case change to the database)**

In [155]:
# Save embeddings to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks)
embeddings_df_save_path = "text_chunks_and_embeddings_abastores.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path,index=False)

**Read the info**

In [156]:
# Import saved file and view
text_chunks_and_embedding_df_load=pd.read_csv(embeddings_df_save_path)
text_chunks_and_embedding_df_load.head()

,sentence_chunk,first_line,chunk_char_count,chunk_word_count,chunk_token_count,mes,anio,producto,provincia,embedding
0,Provincia: Toledo Producto: Avena Fuente de da...,precio: 153.0 fecha: 2002-01-25 precio: 153.0 ...,108,17,27.00,1,2002,Avena,Toledo,[-2.54472345e-02 -1.32305026e-02 -2.87061185e-...
1,Provincia: Toledo Producto: Cebada Fuente de d...,precio: 143.0 fecha: 2002-01-25 precio: 144.0 ...,109,17,27.25,1,2002,Cebada,Toledo,[-2.00621821e-02 -1.33494465e-02 -2.68232096e-...
2,Provincia: Toledo Producto: Maíz Fuente de dat...,precio: 144.0 fecha: 2002-01-25 precio: 144.0 ...,107,17,26.75,1,2002,Maíz,Toledo,[-2.65108049e-02 -2.00800616e-02 -2.67465822e-...
3,Provincia: Toledo Producto: Trigo Fuente de da...,precio: 180.0 fecha: 2002-01-25 precio: 147.0 ...,108,17,27.00,1,2002,Trigo,Toledo,[-1.93464588e-02 -3.79221216e-02 -2.79082097e-...
4,Provincia: Toledo Producto: Avena Fuente de da...,precio: 126.21 fecha: 2003-01-31 precio: 126.2...,109,17,27.25,1,2003,Avena,Toledo,[-1.59097314e-02 -1.62391402e-02 -3.14074904e-...


In [157]:
import random

import torch
import numpy as np
import pandas as pd

device = "cpu"

# Import text and embedding df
text_chunks_and_embedding_df=pd.read_csv(embeddings_df_save_path)

# Convert embedding column back to np.array (it got converted to string when it saved to CSV)
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"),sep=" "))

# Convert our embeddings into a torch.tensor
embeddings = torch.tensor(np.stack(text_chunks_and_embedding_df["embedding"].tolist(),axis=0),dtype=torch.float32).to(device)

# Convert texts and embedding df to list of dicts
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

#text_chunks_and_embedding_df

In [158]:
from sentence_transformers import util, SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device=device) # choose the device to load the model to

**Dot product**

In [184]:
# 1. Define the query
# Note: This could be anything. But since we're working with a nutrition textbook, we'll stick with nutrition-based queries.
query = "precio del maíz para febrero 2023"
print(f"Query: {query}")

# 2. Embed the query to the same numerical space as the text examples
# Note: It's important to embed your query with the same model you embedded your examples with.
query_embedding = embedding_model.encode(query, convert_to_tensor=True).to("cpu")

# 3. Get similarity scores with the dot product (we'll time this for fun)
from time import perf_counter as timer

start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
end_time = timer()

print(f"[INFO] Time take to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

# 4. Get the top-k results (we'll keep this to 5)
top_results_dot_product = torch.topk(dot_scores, k=10)
top_results_dot_product

print("Results:")
# Loop through zipped together scores and indices from torch.topk
for score,idx in zip(top_results_dot_product[0],top_results_dot_product[1]):
  print(f"Score: {score:.4f}")
  print("Text:")
  print(pages_and_chunks[idx]["sentence_chunk"])
  print("\n")

Query: precio del maíz para febrero 2023
[INFO] Time take to get scores on 3917 embeddings: 0.00245 seconds.
Results:
Score: 0.6834
Text:
Provincia: León Producto: Maíz Fuente de datos: Abastores Fecha: mes: 2-febrero año: 2024-dos mil veinticuatro


Score: 0.6805
Text:
Provincia: León Producto: Maíz Fuente de datos: Abastores Fecha: mes: 7-julio año: 2023-dos mil veintitrés


Score: 0.6805
Text:
Provincia: León Producto: Maíz Fuente de datos: Abastores Fecha: mes: 2-febrero año: 2023-dos mil veintitrés


Score: 0.6645
Text:
Provincia: León Producto: Maíz Fuente de datos: Abastores Fecha: mes: 3-marzo año: 2023-dos mil veintitrés


Score: 0.6621
Text:
Provincia: León Producto: Maíz Fuente de datos: Lonja de León Fecha: mes: 2-febrero año: 2024-dos mil veinticuatro


Score: 0.6600
Text:
Provincia: León Producto: Maíz Fuente de datos: Abastores Fecha: mes: 3-marzo año: 2024-dos mil veinticuatro


Score: 0.6589
Text:
Provincia: León Producto: Maíz Fuente de datos: Abastores Fecha: mes: 5-

**Cosine similarity**

In [185]:
# Import cosine_similarity function from sklearn
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity scores
start_time = timer()
cosine_scores = cosine_similarity(query_embedding.reshape(1, -1), embeddings)
end_time = timer()

print(f"[INFO] Time taken to compute cosine similarity scores: {end_time - start_time:.5f} seconds.")

# Convert cosine similarity scores to torch tensor for torch.topk
cosine_scores_tensor = torch.tensor(cosine_scores[0])

# Get the top-k results
top_results_cosine_similarity = torch.topk(cosine_scores_tensor, k=10)

print("Results:")
# Loop through zipped together scores and indices from torch.topk
for score, idx in zip(top_results_cosine_similarity[0], top_results_cosine_similarity[1]):
    print(f"Score: {score:.4f}")
    print("Text:")
    print(pages_and_chunks[idx]["sentence_chunk"])
    print("\n")


[INFO] Time taken to compute cosine similarity scores: 0.02879 seconds.
Results:
Score: 0.6834
Text:
Provincia: León Producto: Maíz Fuente de datos: Abastores Fecha: mes: 2-febrero año: 2024-dos mil veinticuatro


Score: 0.6805
Text:
Provincia: León Producto: Maíz Fuente de datos: Abastores Fecha: mes: 7-julio año: 2023-dos mil veintitrés


Score: 0.6805
Text:
Provincia: León Producto: Maíz Fuente de datos: Abastores Fecha: mes: 2-febrero año: 2023-dos mil veintitrés


Score: 0.6645
Text:
Provincia: León Producto: Maíz Fuente de datos: Abastores Fecha: mes: 3-marzo año: 2023-dos mil veintitrés


Score: 0.6621
Text:
Provincia: León Producto: Maíz Fuente de datos: Lonja de León Fecha: mes: 2-febrero año: 2024-dos mil veinticuatro


Score: 0.6600
Text:
Provincia: León Producto: Maíz Fuente de datos: Abastores Fecha: mes: 3-marzo año: 2024-dos mil veinticuatro


Score: 0.6589
Text:
Provincia: León Producto: Maíz Fuente de datos: Abastores Fecha: mes: 5-mayo año: 2023-dos mil veintitrés


S

**Mixed**

In [168]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 1. Define the query
# Note: This could be anything. But since we're working with a nutrition textbook, we'll stick with nutrition-based queries.
query = "precio del maíz para febrero 2023"
print(f"Query: {query}")

# 2. Embed the query to the same numerical space as the text examples
# Note: It's important to embed your query with the same model you embedded your examples with.
query_embedding = embedding_model.encode(query, convert_to_tensor=True).to("cpu")


# Convert text chunks to list of strings
text_chunk_strings = [item["sentence_chunk"] for item in pages_and_chunks]

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform text chunks to create BoW representation
text_chunk_bow = vectorizer.fit_transform(text_chunk_strings)

# Transform query to BoW representation
query_bow = vectorizer.transform([query])

# Compute cosine similarity between query BoW and text chunks BoW
bow_scores = cosine_similarity(query_bow, text_chunk_bow)[0]

# Weighting factors
dot_weight = 0.7  # Weight for dot product similarity scores
bow_weight = 0.3  # Weight for BoW similarity scores
print('dot_scores:',dot_scores)
print('bow_scores:',bow_scores)

# Combine dot product scores and BoW scores
final_scores = dot_weight * dot_scores + bow_weight * bow_scores

# Get top-k results
top_results_final = torch.topk(final_scores, k=10)

print("Results with combined scores:")
# Loop through zipped together scores and indices from torch.topk
for score, idx in zip(top_results_final[0], top_results_final[1]):
    print(f"Score: {score:.4f}")
    print("Text:")
    print(pages_and_chunks[idx]["sentence_chunk"])
    print("\n")

Query: precio del maíz para febrero 2023
dot_scores: tensor([0.5399, 0.5286, 0.5333,  ..., 0.7536, 0.5686, 0.5852])
bow_scores: [0.         0.         0.11785113 ... 0.12038585 0.12038585 0.12038585]
Results with combined scores:
Score: 0.6363
Text:
Provincia: León Producto: Maíz Fuente de datos: Abastores Fecha: mes: 2-febrero año: 2023-dos mil veintitrés


Score: 0.6346
Text:
Provincia: León Producto: Maíz Fuente de datos: Lonja de León Fecha: mes: 2-febrero año: 2023-dos mil veintitrés


Score: 0.6060
Text:
Provincia: León Producto: Avena Fuente de datos: Lonja de León Fecha: mes: 2-febrero año: 2023-dos mil veintitrés


Score: 0.6042
Text:
Provincia: León Producto: Maíz Fuente de datos: Lonja de León Fecha: mes: 12-diciembre año: 2023-dos mil veintitrés


Score: 0.6017
Text:
Provincia: León Producto: Centeno Fuente de datos: Lonja de León Fecha: mes: 2-febrero año: 2023-dos mil veintitrés


Score: 0.6008
Text:
Provincia: León Producto: Trigo Fuente de datos: Lonja de León Fecha: me

**Semantic search pipeline**

In [277]:
import textwrap
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def print_wrapped(text,wrap_length=80):
  wrapped_text = textwrap.fill(text,wrap_length)
  print(wrapped_text)
    
def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                pages_and_chunks,
                                model: SentenceTransformer=embedding_model,
                                n_resources_to_return: int=5,
                                print_time: bool=True):
    """
    Embeds a query with model and returns top k scores and indices from embeddings.
    """

    # Embed the query
    query_embedding = model.encode(query,
                                   convert_to_tensor=True)
    
    dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
    text_chunk_strings = [item["sentence_chunk"] for item in pages_and_chunks]
    vectorizer = CountVectorizer()
    text_chunk_bow = vectorizer.fit_transform(text_chunk_strings)
    query_bow = vectorizer.transform([query])
    bow_scores = cosine_similarity(query_bow, text_chunk_bow)[0]

    dot_weight = 0.7  # Weight for dot product similarity scores
    bow_weight = 0.3  # Weight for BoW similarity scores
    final_scores = dot_weight * dot_scores + bow_weight * bow_scores

    
    # Get dot product scores on embeddings
    start_time = timer()
    dot_scores = util.dot_score(query_embedding, embeddings)[0]
    end_time = timer()

    if print_time:
        print(f"[INFO] Time taken to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

    scores, indices = torch.topk(input=final_scores,
                                 k=n_resources_to_return)

    return scores, indices

def print_top_results_and_scores(query: str,
                                 embeddings: torch.tensor,
                                 pages_and_chunks: list[dict]=pages_and_chunks,
                                 n_resources_to_return: int=5):
    """
    Takes a query, retrieves most relevant resources and prints them out in descending order.

    Note: Requires pages_and_chunks to be formatted in a specific way (see above for reference).
    """

    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings,pages_and_chunks=pages_and_chunks,
                                                  n_resources_to_return=n_resources_to_return)

    #print(f"Query: {query}\n")
    #print("Results:")
    # Loop through zipped together scores and indicies
    #for score, index in zip(scores, indices):
        #print(f"Score: {score:.4f}")
        # Print relevant sentence chunk (since the scores are in descending order, the most relevant chunk will be first)
        #print_wrapped(pages_and_chunks[index]["sentence_chunk"])
        #print_wrapped(pages_and_chunks[index]["first_line"])
        # Print the page number too so we can reference the textbook further and check the results
        #print("\n")
    return pages_and_chunks[indices[0]]["sentence_chunk"],pages_and_chunks[indices[0]]["first_line"]

In [278]:
query = "precio del maíz para febrero 2023"

# Get just the scores and indices of top related results
scores, indices = retrieve_relevant_resources(query=query,pages_and_chunks=pages_and_chunks,
                                              embeddings=embeddings)
scores, indices

[INFO] Time taken to get scores on 3917 embeddings: 0.00020 seconds.


(tensor([0.6062, 0.5861, 0.5693, 0.5653, 0.5650], dtype=torch.float64),
 tensor([562, 561, 566, 569, 614]))

In [279]:
# Print out the texts of the top scores
topic,text=print_top_results_and_scores(query=query,
                             embeddings=embeddings)

print(topic,text)

[INFO] Time taken to get scores on 3917 embeddings: 0.00021 seconds.
Provincia: León Producto: Maíz Fuente de datos: Abastores Fecha: mes: 2-febrero año: 2023-dos mil veintitrés precio: 303.0 fecha: 2023-02-28 precio: 303.0 fecha: 2023-02-27 precio: 304.0 fecha: 2023-02-24 precio: 304.0 fecha: 2023-02-23 precio: 304.0 fecha: 2023-02-22 precio: 305.0 fecha: 2023-02-21 precio: 305.0 fecha: 2023-02-20 precio: 305.0 fecha: 2023-02-17 precio: 305.0 fecha: 2023-02-16 precio: 306.0 fecha: 2023-02-15 precio: 305.0 fecha: 2023-02-14 precio: 302.75 fecha: 2023-02-13 precio: 301.25 fecha: 2023-02-10 precio: 301.0 fecha: 2023-02-09 precio: 301.0 fecha: 2023-02-08 precio: 301.0 fecha: 2023-02-07 precio: 300.0 fecha: 2023-02-06 precio: 300.0 fecha: 2023-02-03 precio: 300.25 fecha: 2023-02-02 precio: 301.0 fecha: 2023-02-01


In [280]:
text

'precio: 303.0 fecha: 2023-02-28 precio: 303.0 fecha: 2023-02-27 precio: 304.0 fecha: 2023-02-24 precio: 304.0 fecha: 2023-02-23 precio: 304.0 fecha: 2023-02-22 precio: 305.0 fecha: 2023-02-21 precio: 305.0 fecha: 2023-02-20 precio: 305.0 fecha: 2023-02-17 precio: 305.0 fecha: 2023-02-16 precio: 306.0 fecha: 2023-02-15 precio: 305.0 fecha: 2023-02-14 precio: 302.75 fecha: 2023-02-13 precio: 301.25 fecha: 2023-02-10 precio: 301.0 fecha: 2023-02-09 precio: 301.0 fecha: 2023-02-08 precio: 301.0 fecha: 2023-02-07 precio: 300.0 fecha: 2023-02-06 precio: 300.0 fecha: 2023-02-03 precio: 300.25 fecha: 2023-02-02 precio: 301.0 fecha: 2023-02-01'

In [281]:
from openai import OpenAI

client=OpenAI(api_key='')

query = "precio del maíz para febrero 2023"

topic,text=print_top_results_and_scores(query=query,
                             embeddings=embeddings)

stream = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "assistant", "content": "Eres un experto parte de una organizacion llamada Abastores que se encarga de responder preguntas de usuario relacionadas al precio de ciertos granos que se venden en tu plataforma"},
        {"role": "system", "content": f"Toda pregunta debe ser respondida en base a la siguiente data:"},
        {"role": "system", "content": f"La metadata de la data que te brindo debajo es la siguiente: {topic}"},
        {"role": "system", "content": f"Esta es la data: {text}"},
        {"role": "system", "content": f"La pregunta del usuario es esta:{query}"},
    ],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")


[INFO] Time taken to get scores on 3917 embeddings: 0.00023 seconds.
El precio del maíz en la provincia de León durante el mes de Febrero 2023 osciló entre 300.0 y 306.0. Aquí algunos datos clave:

- El precio más bajo fue de 300.0, registrado en las fechas 2023-02-06 y 2023-02-03.
- El precio más alto fue de 306.0, registrado en la fecha 2023-02-15.
- Al final del mes, el 28 de Febrero 2023, el precio era de 303.0. 

Es importante notar que estos precios pueden variar durante el mes y la información proporcionada fue recopilada de nuestra fuente de datos "Abastores".